<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Airline Tweets Sentiment Analysis Lab

_Authors: Phillippa Thomson (NYC)_

---

You are going to be analyzing tweets about airlines.  These have been hand-tagged with sentiment.  There are three categories: positive, neutral, and negative.

Use VADER to calculate sentiment for each tweet, and see if you can correctly predict the hand-tagged sentiment.

What is the accuracy?  Print out a heatmap to see where your model performs well, and where it performs poorly.

In [15]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, \
precision_score, recall_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
tweets = pd.read_csv("./dataset/Tweets.csv")

In [23]:
tweets.head()

,airline_sentiment,airline,text
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...


### 1. Preview the airline_sentiment column.
- What percentage of reviews are positive, neutral, and negative?

In [18]:
# A:
tweets.airline_sentiment.value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

### 2. Load in the Sentiment IntensityAnalyzer from Vader and add compound, negative, neutral, and positive scores into the DataFrame.

In [26]:
# A:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [31]:
vs = analyzer.polarity_scores(tweets.text[0])
vs

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [32]:
# A:
tweets['pos'] = 0
tweets['neg'] = 0
tweets['neu'] = 0
tweets['comp'] = 0

for index, tweet in enumerate(tweets.text.values):
    vs = analyzer.polarity_scores(tweet)
    tweets.loc[index,'pos'] = vs['pos']
    tweets.loc[index,'neg'] = vs['neg']
    tweets.loc[index,'neu'] = vs['neu']
    tweets.loc[index,'comp'] = vs['compound']

In [48]:
tweets.airline_sentiment = tweets.airline_sentiment.map(lambda x: 0 if x == 'neutral' else 1 if x == 'positive' else 2)

### 3. Store airline_sentiment in y to use as labels and create an appropriate feature matrix, X.

In [49]:
# A:
y = tweets.airline_sentiment.values
X = tweets[['pos','neg','neu','comp']]

### 4. Fit a model of your choice to predict airline_sentient and cross-validate.

In [50]:
# A:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [52]:
# A:
ss = StandardScaler()
Xs = ss.fit_transform(X)

logcv_score = cross_val_score(LogisticRegression(), Xs, y, cv = 10)
print 'mean logcv score:{}'.format(np.mean(logcv_score))

baseline_score = float(tweets.airline_sentiment.value_counts()[2])/np.sum(tweets.airline_sentiment.value_counts())

print 'baseline score:{}'.format(baseline_score)

mean logcv score:0.694469759473
baseline score:0.626912568306


In [60]:
# A:
logreg = LogisticRegression()
logreg.fit(Xs, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
# A:

### 5. Display the confusion matrix.
- What reviews are difficult to identify?

In [57]:
logreg.predict(X)

array([2, 2, 2, ..., 2, 2, 2])

In [62]:
# A:
from sklearn.metrics import confusion_matrix

confu_matrix = confusion_matrix(y, logreg.predict(Xs))
confu_matrix

array([[  11,  467, 2621],
       [   2, 1405,  956],
       [   4,  430, 8744]])

### 6. Print the classification report and discuss the characteristics of the model.

In [ ]:
# A:

The model does ok with negative tweets (the predominant class) but quite poorly with neutral.

To put this in perspective, human concordance, the probability that two people assign the same sentiment to an observation is usually around 70%-80% our baseline is at 63%. Even small increases in accuracy quickly move us towards a theoretical maximum in performance.